In [1]:
import dataflowkit.datasets as D
import dataflowkit.graphs as G
import dataflowkit.recipes as R
from dataflowkit.utils.print_time import print_time, print_end
from multiprocessing import Process, Queue, cpu_count
import threading, numpy as np

In [2]:
class MemoryRecipe(R.BaseRecipe):
    # Must be executed in singleton, maybe heavy memory demanding
    def execute(self, ins, outs):
        # Takes ~ 0.093899s
        for i in np.arange(1e6):
            pass

In [3]:
class CPURecipe(R.BaseRecipe):
    # Hevay calcuation process
    def execute(self, ins, outs):
        # Takes ~ 0.823933s
        for i in np.arange(1e7):
            pass

In [4]:
def thread_job(file_queue, mrf, master_index):
    while not file_queue.empty():
        file = file_queue.get()
        print_time('master_index: ' + str(master_index) + ' file: ' + str(file))
        
        # declare recipes
        dummy_recipe1 = MemoryRecipe()
        dummy_recipe2 = mrf.create(CPURecipe, no_slaves=4)
        
        # declare datasets
        dataset1 = D.InMemory()
        dataset2 = D.InMemory()
        dataset3 = D.InMemory()
        
        # declare graph
        dummy_recipe1.execute(ins=dict(dataset1=dataset1), outs=dict(dataset2=dataset2))
        dummy_recipe2.execute(ins=dict(dataset2=dataset2), outs=dict(dataset3=dataset3))
        
        print_end('master_index: ' + str(master_index) + ' file: ' + str(file))

#### Declare the files be process

In [5]:
file_queue = Queue()
files = np.arange(20)
for file in files:
    file_queue.put(file)

#### MasterRecipeFactory must be initialized before opeining the processes

In [6]:
no_master = 4
mrf = G.MasterRecipeFactory.get_inatance()

#### Declare the threads

In [7]:
threads = [threading.Thread(target=thread_job, args=(file_queue, mrf, master_index)) 
           for master_index in np.arange(no_master)]

#### Execute the threads

In [8]:
for t in threads:
    t.start()
for t in threads:
    t.join()

master_index: 0 file: 0 1.71856
master_index: 3 file: 3 1.721976
master_index: 1 file: 1 1.731497
master_index: 2 file: 2 1.791534
master_index: 3 file: 5 1.340705
master_index: 0 file: 4 1.381528
master_index: 1 file: 6 1.456624
master_index: 2 file: 7 1.410798
master_index: 3 file: 8 1.291731
master_index: 0 file: 9 1.316931
master_index: 1 file: 10 1.329325
master_index: 2 file: 11 1.47841
master_index: 3 file: 12 1.381444
master_index: 0 file: 13 1.394525
master_index: 1 file: 14 1.439539
master_index: 2 file: 15 1.423967
master_index: 3 file: 16 1.287339
master_index: 0 file: 17 1.253891
master_index: 1 file: 18 1.218341
master_index: 2 file: 19 1.102599
